In [1]:
import pandas as pd
import numpy as np
import re

In [3]:
patients_df=pd.read_csv('patients.csv')
treatments_df=pd.read_csv('treatments.csv')
treatments_cut_df=pd.read_csv('treatments_cut.csv')
adverse_reactions_df=pd.read_csv('adverse_reactions.csv')

In [4]:
patients_df.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [5]:
treatments_cut_df.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,NaN
2,alwin,svensson,36u - 39u,-,7.78,7.34,NaN
3,thể,lương,-,61u - 64u,7.64,7.22,0.92
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38


In [6]:
treatments_df.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [7]:
adverse_reactions_df.head()

,given_name,surname,adverse_reaction
0,berta,napolitani,injection site discomfort
1,lena,baer,hypoglycemia
2,joseph,day,hypoglycemia
3,flavia,fiorentino,cough
4,manouck,wubbels,throat irritation


## DATA SUMMARY






This is the data of 500 patients in which 300 participated in clinical trials , initally no one were using Novodra(injection based insulin injector ) or auralin (oral based insulin injector (on researched) ). All were experiencing high HBA1C level

All 350 patient were first injected with novodra to maintain baseline HBA1C level , After 4 weeks

175 patients switched to auralin for 24 weeks
175 patients switched to novodra for 24 weeks

to notice the change in hba1c level


patients who were facing adverse reaction are also recorded

## *DATA ASSEEMENT*



```
- MANUAL (by watching data in  excel or google sheet etc)
- PROGRAMMATIC  (Using pandas functions)
```



In [8]:
with pd.ExcelWriter('clinical_trial.xlsx') as writer:
    patients_df.to_excel(writer, sheet_name='patients')
    treatments_df.to_excel(writer, sheet_name='treatments')
    treatments_cut_df.to_excel(writer, sheet_name='treatments_cut')
    adverse_reactions_df.to_excel(writer, sheet_name='adverse_reactions')

## *ISSUES WITH THE DATA*

# 1. Dirty data

   Table  - `patients`

- In the state col , some are in full form some are in short form
- missing values in address , city , state , zipcode , country , contact
- incorrect data type is assigned to zipCode , birthdate col
- there are 5 duplicated rows by the name john doe
- one patients has weight 48 pound (21 kg)
- one patient has height 27 inches


  Table - `treatment` & `treatment_cut`

- name and surname are all in lower case
-  'u' should be removed from auralin and novodra
- there are missing value in hba1c change column
- can make better  hba1c change col by  hba1c start - hba1c end

# 2. messy data

  Table - `Patients`
- In the contact col , contact no and address are merge , have to seperate them

    Table `treatment` & `tretment_cut`

- Auraling and novodra should be split into two cols start and end dose

- have to merge treatment and treatment_cut

    Table `adverse reaction`

- this table is not making sense , beacuse it is only created for recording adverse reaction , which can be easily come in treatment table      





## DATA CLEANING

definine:

    table `patient`
   - handled missing values in address , city , state , zipcode , country , contact

   - seperated contact no and address from the column contact




In [97]:
## copying data

patients=patients_df.copy()
treatments=treatments_df.copy()
treatments_cut=treatments_cut_df.copy()
adverse_reactions=adverse_reactions_df.copy()

In [98]:
patients

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,CA,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,IL,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,NE,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,ME,3852.0,United States,207-477-0579MustafaLindstrom@jourrapide.com,4/10/1959,181.1,72,24.6
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,AZ,86341.0,United States,928-284-4492RumanBisliev@gustr.com,3/26/1948,239.6,70,34.4
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,MO,64110.0,United States,816-223-6007JinkedeKeizer@teleworm.us,1/13/1971,171.2,67,26.8
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,WA,98109.0,United States,ChidaluOnyekaozulu@jourrapide.com1 360 443 2060,2/13/1952,176.9,67,27.7


In [99]:
patients[patients['address'].isnull()]

patients.fillna('No data', inplace=True)

<ipython-input-99-bab925f4994b>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'No data' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  patients.fillna('No data', inplace=True)


In [100]:
patients.isnull().sum()

,0
patient_id,0
assigned_sex,0
given_name,0
surname,0
address,0
city,0
state,0
zip_code,0
country,0
contact,0


In [101]:
patients['contact'].head(10)

,contact
0,951-719-9170ZoeWellish@superrito.com
1,PamelaSHill@cuvox.de+1 (217) 569-3204
2,402-363-6804JaeMDebord@gustr.com
3,PhanBaLiem@jourrapide.com+1 (732) 636-8246
4,334-515-7487TimNeudorf@cuvox.de
5,386-334-5237RafaelCardosoCosta@gustr.com
6,775-533-5933MaryBAdams@einrot.com
7,XiuxiuChang@einrot.com1 408 778 3236
8,816-265-9578DavidGustafsson@armyspy.com
9,SophieCabreraIbarra@teleworm.us1 718 795 9124


In [102]:
## Updated code as - code used in the session using append on dataframe, which is now deprecated - removed.

# Initialize a list to collect the data
data = []

# Iterate through the data
for item in patients['contact']:
    # Convert item to a string to ensure compatibility with re.search
    item = str(item)

    # Use regular expressions to find the phone numbers and email addresses
    phone_match = re.search(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})', item)
    phone = phone_match.group(0) if phone_match else None

    # Remove the phone number from the item
    item = re.sub(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})', '', item)

    email_match = re.search(r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', item)
    email = email_match.group(0) if email_match else None

    data.append({'phone': phone, 'email': email})

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['phone', 'email'])

print(df)

              phone                               email
0      951-719-9170            ZoeWellish@superrito.com
1    (217) 569-3204                PamelaSHill@cuvox.de
2      402-363-6804                JaeMDebord@gustr.com
3    (732) 636-8246           PhanBaLiem@jourrapide.com
4      334-515-7487                 TimNeudorf@cuvox.de
..              ...                                 ...
498    207-477-0579     MustafaLindstrom@jourrapide.com
499    928-284-4492              RumanBisliev@gustr.com
500    816-223-6007           JinkedeKeizer@teleworm.us
501    360 443 2060  ChidaluOnyekaozulu@jourrapide.com1
502    402-848-4923            PatrickGersten@rhyta.com

[503 rows x 2 columns]


In [103]:
patients['phone'] = patients["contact"].apply(lambda x: find_contact_details(x)).apply(lambda x:x[0])
patients['email'] = patients["contact"].apply(lambda x: find_contact_details(x)).apply(lambda x:x[1])

In [104]:
patients.head(5)

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,CA,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6,"(, 951-719-9170)",ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,IL,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2,"(+1 , (217) 569-3204)",PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,NE,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8,"(, 402-363-6804)",JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7,"(+1 , (732) 636-8246)",PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1,"(, 334-515-7487)",TimNeudorf@cuvox.de


In [105]:
patients.drop(columns=['contact'],inplace=True)

In [106]:
patients.head(5)

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,CA,92390.0,United States,7/10/1976,121.7,66,19.6,"(, 951-719-9170)",ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,IL,61812.0,United States,4/3/1967,118.8,66,19.2,"(+1 , (217) 569-3204)",PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,NE,68467.0,United States,2/19/1980,177.8,71,24.8,"(, 402-363-6804)",JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,7/26/1951,220.9,70,31.7,"(+1 , (732) 636-8246)",PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,2/18/1928,192.3,27,26.1,"(, 334-515-7487)",TimNeudorf@cuvox.de


In [107]:
patients['phone']

,phone
0,"(, 951-719-9170)"
1,"(+1 , (217) 569-3204)"
2,"(, 402-363-6804)"
3,"(+1 , (732) 636-8246)"
4,"(, 334-515-7487)"
...,...
498,"(, 207-477-0579)"
499,"(, 928-284-4492)"
500,"(, 816-223-6007)"
501,"(, 360 443 2060)"


In [108]:
patients.head(9)

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,CA,92390.0,United States,7/10/1976,121.7,66,19.6,"(, 951-719-9170)",ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,IL,61812.0,United States,4/3/1967,118.8,66,19.2,"(+1 , (217) 569-3204)",PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,NE,68467.0,United States,2/19/1980,177.8,71,24.8,"(, 402-363-6804)",JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,7/26/1951,220.9,70,31.7,"(+1 , (732) 636-8246)",PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,2/18/1928,192.3,27,26.1,"(, 334-515-7487)",TimNeudorf@cuvox.de
5,6,male,Rafael,Costa,1140 Willis Avenue,Daytona Beach,FL,32114.0,United States,8/31/1931,183.9,70,26.4,"(, 386-334-5237)",RafaelCardosoCosta@gustr.com
6,7,female,Mary,Adams,3145 Sheila Lane,Burbank,NV,84728.0,United States,11/19/1969,146.3,65,24.3,"(, 775-533-5933)",MaryBAdams@einrot.com
7,8,female,Xiuxiu,Chang,2687 Black Oak Hollow Road,Morgan Hill,CA,95037.0,United States,8/13/1958,158.0,60,30.9,"(, 408 778 3236)",XiuxiuChang@einrot.com1
8,9,male,Dsvid,Gustafsson,1790 Nutter Street,Kansas City,MO,64105.0,United States,3/6/1937,163.9,66,26.5,"(, 816-265-9578)",DavidGustafsson@armyspy.com


In [109]:
# Issue 2 : state col sometimes contain full name and some times abbrivietation consistency

patients.state.value_counts()

,count
state,
CA,60
NY,47
TX,32
IL,24
FL,22
MA,22
PA,18
GA,15
OH,14


In [110]:
state_abbreviations = {
    'California': 'CA',
    'Texas': 'TX',
    'New York': 'NY',
    'Massachusetts': 'MA',
    'Pennsylvania': 'PA',
    'Georgia': 'GA',
    'Illinois': 'IL',
    'Ohio': 'OH',
    'Florida': 'FL',
    'Michigan': 'MI',
    'Oklahoma': 'OK',
    'Louisiana': 'LA',
    'New Jersey': 'NJ',
    'Virginia': 'VA',
    'Mississippi': 'MS',
    'Wisconsin': 'WI',
    'Indiana': 'IN',
    'Minnesota': 'MN',
    'Tennessee': 'TN',
    'Alabama': 'AL',
    'North Carolina': 'NC',
    'Kentucky': 'KY',
    'Washington': 'WA',
    'Missouri': 'MO',
    'Idaho': 'ID',
    'Kansas': 'KS',
    'Nevada': 'NV',
    'South Carolina': 'SC',
    'Iowa': 'IA',
    'Connecticut': 'CT',
    'Maine': 'ME',
    'North Dakota': 'ND',
    'Nebraska': 'NE',
    'Rhode Island': 'RI',
    'Arkansas': 'AR',
    'Colorado': 'CO',
    'Arizona': 'AZ',
    'Maryland': 'MD',
    'Delaware': 'DE',
    'West Virginia': 'WV',
    'Oregon': 'OR',
    'South Dakota': 'SD',
    'Montana': 'MT',
    'Vermont': 'VT',
    'District of Columbia': 'DC',
    'Alaska': 'AK',
    'Wyoming': 'WY',
    'New Hampshire': 'NH',
    'New Mexico': 'NM',
    'No data': 'Unknown'
}

# Apply changes to the DataFrame
patients['state'] = patients['state'].apply(lambda x: state_abbreviations.get(x, x))

# Final check of the dataframe
print(patients['state'].value_counts())


state
CA         60
NY         47
TX         32
IL         24
FL         22
MA         22
PA         18
GA         15
OH         14
MI         13
OK         13
LA         13
NJ         12
Unknown    12
VA         11
WI         10
MS         10
IN          9
MN          9
TN          9
AL          9
WA          8
KY          8
NC          8
MO          7
NV          6
KS          6
ID          6
NE          6
SC          5
CT          5
IA          5
CO          4
ND          4
RI          4
ME          4
AR          4
AZ          4
SD          3
MD          3
WV          3
OR          3
DE          3
VT          2
MT          2
DC          2
WY          1
AK          1
NH          1
NM          1
Name: count, dtype: int64


In [112]:
patients=patients.drop_duplicates(subset=['given_name','surname'])

In [114]:
patients.duplicated(subset=['given_name','surname']).sum()

0